In [1]:
import pandas as pd
import numpy as np
import time

In [ ]:
# raw_dt= pd.read_csv('data/202107-citibike-tripdata_pre_analyzed.csv')
# station_cluster = pd.read_csv('intersection_station_cluster.csv') 
# cluster_dict = dict(zip(station_cluster['station_id'], station_cluster['cluster']))

In [ ]:
# month_dt= raw_dt[(raw_dt['start_station_id'].isin(station_cluster['station_id']) )& (raw_dt['end_station_id'].isin(station_cluster['station_id']))]
# month_dt['start_station_id']= month_dt['start_station_id'].map(cluster_dict)
# month_dt['end_station_id']= month_dt['end_station_id'].map(cluster_dict)
# date_ls= sorted(list(month_dt['started_date'].unique())) 

In [ ]:
paths = ['data/preprocessed/202107-citibike-tripdata_pre_analyzed.csv',
        'data/preprocessed/202108-citibike-tripdata_pre_analyzed.csv',
        'data/preprocessed/202109-citibike-tripdata_pre_analyzed.csv']

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
print(os.getcwd())
os.chdir('/content/drive/MyDrive/3001_Proj/preprocessed')
paths = os.listdir(os.getcwd())
for p in paths: print(p)

/content
intersection_station_cluster.csv
202108-citibike-tripdata_pre_analyzed.csv
202102-citibike-tripdata_pre_analyzed.csv
202111-citibike-tripdata_pre_analyzed.csv
202105-citibike-tripdata_pre_analyzed.csv
202106-citibike-tripdata_pre_analyzed.csv
202109-citibike-tripdata_pre_analyzed.csv
202112-citibike-tripdata_pre_analyzed.csv
202103-citibike-tripdata_pre_analyzed.csv
202110-citibike-tripdata_pre_analyzed.csv
202104-citibike-tripdata_pre_analyzed.csv
202107-citibike-tripdata_pre_analyzed.csv
hour_csv


In [12]:
from tqdm import tqdm
from google.colab import files
def create_hour_csv(path):
    
    raw_dt= pd.read_csv(path)
    station_cluster = pd.read_csv('intersection_station_cluster.csv') 
    cluster_dict = dict(zip(station_cluster['station_id'], 
                            station_cluster['cluster']))
    
    month_dt= raw_dt[(raw_dt['start_station_id'].isin(station_cluster['station_id']) )& (raw_dt['end_station_id'].isin(station_cluster['station_id']))]
    month_dt['start_station_id']= month_dt['start_station_id'].map(cluster_dict)
    month_dt['end_station_id']= month_dt['end_station_id'].map(cluster_dict)
    date_ls= sorted(list(month_dt['started_date'].unique())) 
    
    ##############################################################################
    
    month_matrix=[]
    hour_ls=[]
    full_date_ls=[]
    
    
    for date in date_ls:
        date_dt= month_dt[month_dt['started_date']==date]
        for hour in np.arange(start=0, stop= 24, step=2):
            hour_dt= date_dt[(date_dt['started_hour']<=hour+1) 
                             & (date_dt['started_hour']>= hour) 
                             &(date_dt['ended_hour']<=(hour+2))]
            count_hour_dt= hour_dt.groupby(by= ['start_station_id', 'end_station_id'], as_index=False)['ride_id'].count()
            for i in range(6):
                if (i not in set(count_hour_dt['start_station_id'])) or (i not in set(count_hour_dt['end_station_id'])):
                    count_hour_dt = count_hour_dt.append({'start_station_id':i, 'end_station_id':i, 'ride_id':-1},ignore_index=True)
            hour_pivot_dt= count_hour_dt.pivot(index='start_station_id', columns='end_station_id')['ride_id'].fillna(0)

            hour_pivot_dt[hour_pivot_dt<0] = 0
            hour_ls.append(hour)
            full_date_ls.append(date)
            month_matrix.append(hour_pivot_dt)
            hour_pivot_dt.to_csv('./hour_csv/{d}-{h}.csv'.format(d = date, h=hour))



In [8]:
%load_ext autoreload
%autoreload 2
from multiprocessing import Pool
# from helper1 import transformation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Original Version

In [ ]:
start = time.time()

for p in paths:
  if p[0] =="2":
    create_hour_csv(p)

end = time.time()
print(end-start)

C:\ANACONDA\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-3-0955caa40b53>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_dt['start_station_id']= month_dt['start_station_id'].map(cluster_dict)
<ipython-input-3-0955caa40b53>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_dt['end_station_id']= month_dt['end_station_

26.437243223190308


In [14]:
start = time.time()
paths = [p for p in paths if p[0]=="2"]
with Pool(8) as pool: 
    re= pool.map(create_hour_csv, paths)
end = time.time()  

print(end-start)

/usr/lib/python3.7/multiprocessing/pool.py:44: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  return list(map(*args))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/lib/python3.7/multiprocessing/pool.

180.3820035457611


In [39]:
!zip -r /content/drive/MyDrive/3001_Proj/preprocessed/hour_csv.zip /content/drive/MyDrive/3001_Proj/preprocessed/hour_csv

  adding: content/drive/MyDrive/3001_Proj/preprocessed/hour_csv/ (stored 0%)
  adding: content/drive/MyDrive/3001_Proj/preprocessed/hour_csv/2021-06-29-2.csv (deflated 56%)
  adding: content/drive/MyDrive/3001_Proj/preprocessed/hour_csv/2021-09-25-22.csv (deflated 51%)
  adding: content/drive/MyDrive/3001_Proj/preprocessed/hour_csv/2021-04-16-16.csv (deflated 47%)
  adding: content/drive/MyDrive/3001_Proj/preprocessed/hour_csv/2021-06-29-4.csv (deflated 55%)
  adding: content/drive/MyDrive/3001_Proj/preprocessed/hour_csv/2021-04-16-18.csv (deflated 49%)
  adding: content/drive/MyDrive/3001_Proj/preprocessed/hour_csv/2021-04-16-20.csv (deflated 51%)
  adding: content/drive/MyDrive/3001_Proj/preprocessed/hour_csv/2021-06-29-6.csv (deflated 52%)
  adding: content/drive/MyDrive/3001_Proj/preprocessed/hour_csv/2021-04-16-22.csv (deflated 52%)
  adding: content/drive/MyDrive/3001_Proj/preprocessed/hour_csv/2021-09-26-0.csv (deflated 51%)
  adding: content/drive/MyDrive/3001_Proj/preprocessed

In [40]:
files.download('/content/drive/MyDrive/3001_Proj/preprocessed/hour_csv.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
exit()

In [ ]:
# for i in range(7):
#     if i not in set(count_hour_dt['start_station_id'].unique()):
#         count_hour_dt = count_hour_dt.append({'start_station_id':i, 'end_station_id':i, 'ride_id':-2},ignore_index=True)
# count_hour_dt
# hour_pivot_dt= count_hour_dt.pivot(index='start_station_id', columns='end_station_id')['ride_id'].fillna(0)
# hour_pivot_dt